In [1]:
import pandas as pd

Traemos la data necesaria

In [2]:
df_games = pd.read_parquet("../Datasets/steam_games_limpio.parquet")
df_reviews = pd.read_parquet("../Datasets/reviews_analisis_sentiment.parquet")
df_items = pd.read_parquet("../Datasets/australian_items_limpio.parquet")

In [3]:
df_games.head(2)

,publisher,genres,title,reviews_url,price,early_access,id,developer,year
0,Kotoshiro,Action,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018
0,Kotoshiro,Casual,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018


In [4]:
df_reviews.head(2)

,user_id,user_url,reviews_item_id,reviews_recommend,reviews_date,year,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011-11-05,2011,2
1,js41637,http://steamcommunity.com/id/js41637,251610,True,2014-06-24,2014,2


In [5]:
df_items.head(2)

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0


Seleccionamos las columnas necesarias para crear la data para la funcion

Games

In [6]:
df_games = df_games[["id", "developer"]]

Reviews

In [7]:
df_reviews = df_reviews[["user_id", "reviews_item_id", "reviews_recommend"]]

Combinamos los dataframes

In [8]:
df_combined = pd.merge(df_games, df_reviews, left_on="id", right_on="reviews_item_id", how="inner")

In [9]:
df_combined

,id,developer,user_id,reviews_item_id,reviews_recommend
0,70,Valve,EizanAratoFujimaki,70,True
1,70,Valve,76561198020928326,70,True
2,70,Valve,Mutivate,70,True
3,70,Valve,brimmers,70,True
4,70,Valve,76561198065827553,70,True
...,...,...,...,...,...
100224,80,Valve,gummyfrog,80,False
100225,80,Valve,76561198016564178,80,True
100226,80,Valve,kanzabrand,80,True
100227,80,Valve,josehobold,80,True


Items

In [10]:
df_items = df_items[["user_id", "item_id"]]

Creamos la funcion

In [11]:
def sentiment_analysis(desarrolladora: str):
    # filtra juego para desarrollador dado
    games_developer = df_combined[df_combined["developer"]== desarrolladora]
    # obtiene el id de los juegos seleccionados
    game_id = games_developer["id"]
    
    items_developer = df_items[df_items["item_id"].isin(game_id)]
    
    # Obtenemos el id d los usuarios
    user_id = items_developer["user_id"]
    
    reviews_developer = df_combined[df_combined["user_id"].isin(user_id)]
    
    reviews_positive = reviews_developer[reviews_developer["reviews_recommend"] == True].shape[0]
    reviews_negative = reviews_developer[reviews_developer["reviews_recommend"]== False].shape[0]
    
    return {desarrolladora: {"Positive":reviews_positive, "Negative":reviews_negative}}

Prueba de la funcion

In [12]:
sentiment_analysis("Valve")

{'Valve': {'Positive': 16047, 'Negative': 1949}}

Cargamos los datasets

In [13]:
df_items.to_parquet("../Funciones/data/sentiment_analysis_items.parquet")

In [14]:
df_combined.to_parquet("../Funciones/data/sentiment_analysis.parquet")